In [1]:
import numpy as np
import pandas as pd
import os
import joblib

In [2]:
random_state = 100
cv_folds = 5

if not os.path.isdir("model_dumps"):
    os.makedirs("model_dumps")

## Data Loading

Set the filename here

In [3]:
CSV_FILENAME = "final_eda_features.csv" #<-- update csv name
df_features = pd.read_csv(CSV_FILENAME)
df_features.head()

,Unnamed: 0.1,Unnamed: 0,path,updated_source_h,updated_source_w,face_index,used_mask,R_BIN_mask_0,R_BIN_mask_1,R_BIN_mask_2,...,SOBEL_BIN_bbox_14,SOBEL_BIN_bbox_15,SOBEL_BIN_bbox_16,SOBEL_BIN_bbox_17,SOBEL_BIN_bbox_18,SOBEL_BIN_bbox_19,x,y,w,h
0,0,0,C:\Users\jayba\Documents\GitHub\THS-ST1\images...,683,1024,75,True,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.824519,0.508413,0.019231,0.021635
1,1,2,C:\Users\jayba\Documents\GitHub\THS-ST1\images...,768,1024,23,True,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.284856,0.522837,0.012019,0.016827
2,2,4,C:\Users\jayba\Documents\GitHub\THS-ST1\images...,768,1024,1,True,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.918269,0.612981,0.009615,0.014423
3,3,5,C:\Users\jayba\Documents\GitHub\THS-ST1\images...,683,1024,2,True,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.602163,0.334135,0.055288,0.072115
4,4,6,C:\Users\jayba\Documents\GitHub\THS-ST1\images...,632,1024,25,True,0.0,0.166667,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.335337,0.486779,0.016827,0.021635


```
df_features = df_features.drop(columns = ['Unnamed: 0', 'path', 'source_w', 'source_h', 'face_index'])
df_features = df_features.loc[df_features["e_bbox_yf"] < 3, :]
print(df_features.columns)
```

Change string types to numeric types

In [4]:
from sklearn.preprocessing import LabelEncoder

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [5]:
encoder = LabelEncoder()
categorical_columns = df_features.select_dtypes(include=[bool, object]).columns
encoded_columns = df_features[categorical_columns].apply(encoder.fit_transform)
encoded_columns

,path,used_mask,source_file
0,4300,1,10169
1,4301,1,10170
2,4302,1,10171
3,4303,1,10172
4,4304,1,10173
...,...,...,...
21592,9706,1,1030
21593,9707,1,1031
21594,9708,1,1032
21595,9709,1,1033


In [6]:
df_encoded_features = df_features.copy()
df_encoded_features[categorical_columns] = encoded_columns
df_encoded_features

,Unnamed: 0.1,Unnamed: 0,path,updated_source_h,updated_source_w,face_index,used_mask,R_BIN_mask_0,R_BIN_mask_1,R_BIN_mask_2,...,SOBEL_BIN_bbox_14,SOBEL_BIN_bbox_15,SOBEL_BIN_bbox_16,SOBEL_BIN_bbox_17,SOBEL_BIN_bbox_18,SOBEL_BIN_bbox_19,x,y,w,h
0,0,0,4300,683,1024,75,1,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.824519,0.508413,0.019231,0.021635
1,1,2,4301,768,1024,23,1,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.284856,0.522837,0.012019,0.016827
2,2,4,4302,768,1024,1,1,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.918269,0.612981,0.009615,0.014423
3,3,5,4303,683,1024,2,1,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.602163,0.334135,0.055288,0.072115
4,4,6,4304,632,1024,25,1,0.00000,0.166667,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.335337,0.486779,0.016827,0.021635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,21597,27313,9706,233,191,0,1,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.519231,0.574519,0.408654,0.663462
21593,21598,27314,9707,500,366,0,1,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.518029,0.554087,0.367788,0.531250
21594,21599,27315,9708,337,286,0,1,0.22905,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.509615,0.450721,0.432692,0.588942
21595,21600,27316,9709,235,215,0,1,0.00000,0.000000,0.106667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.474760,0.486779,0.362981,0.507212


Split into X and Y

In [7]:
feat_regions = ["bbox", "mask"]

color_channels = {
    "RGB": ("R_BIN_", "G_BIN_", "B_BIN_"),
    "HSV": ("H_HSV_BIN_", "S_HSV_BIN_", "V_HSV_BIN_"),
    "HSL": ("H_HSL_BIN_", "S_HSL_BIN_", "L_HSL_BIN_"),
    "LAB": ("L_LAB_BIN_", "A_LAB_BIN_", "B_LAB_BIN_"),
    "YCBCR": ("Y_BIN_", "CR_BIN_", "CB_BIN_"),
}

#labels = {"bbox": "e_bbox_yf", "mask": "e_face_yf"}
labels = {"mask": "e_bbox_yf", "bbox": "e_face_yf"} #<-- inverted

In [8]:
def get_features_and_label(color_space, region):
    features = ["w", "h", "x", "y"]
    for color_channel in color_channels[color_space]: 
        features += [color_channel + region + "_" + str(i) for i in range(26)]
    features += ["LBP_BIN_" + region + "_" + str(i) for i in range(26)]
    features += ["SOBELX_BIN_" + region + "_" + str(i) for i in range(20)]
    features += ["SOBELY_BIN_" + region + "_" + str(i) for i in range(20)]
    features += ["SOBEL_BIN_" + region + "_" + str(i) for i in range(20)]
    return features, labels[region]

In [9]:
ASSIGNED_COLOR_SPACE = "HSV" #<-- pick a colorspace

In [10]:
features, label = get_features_and_label(ASSIGNED_COLOR_SPACE, "mask")
X_features_mask =  df_encoded_features.loc[:,  features]
y_features_mask = df_encoded_features.loc[:, label].values  #<-- pick label

features, label = get_features_and_label(ASSIGNED_COLOR_SPACE, "bbox")
X_features_bbox =  df_encoded_features.loc[:,  features]
y_features_bbox = df_encoded_features.loc[:, label].values  #<-- pick label

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train_mask, X_test_mask, y_train_mask, y_test_mask = train_test_split(X_features_mask, y_features_mask, test_size = 0.2, random_state=random_state)

print("Split shapes")
print("X_train: ", X_train_mask.shape)
print("y_train: ", y_train_mask.shape)
print("X_test: ", X_test_mask.shape)
print("y_test: ", y_test_mask.shape)

Split shapes
X_train:  (17277, 168)
y_train:  (17277,)
X_test:  (4320, 168)
y_test:  (4320,)


In [13]:
X_train_bbox, X_test_bbox, y_train_bbox, y_test_bbox = train_test_split(X_features_bbox, y_features_bbox, test_size = 0.2, random_state=random_state)

print("Split shapes")
print("X_train: ", X_train_bbox.shape)
print("y_train: ", y_train_bbox.shape)
print("X_test: ", X_test_bbox.shape)
print("y_test: ", y_test_bbox.shape)

Split shapes
X_train:  (17277, 168)
y_train:  (17277,)
X_test:  (4320, 168)
y_test:  (4320,)


## Base Models

### Model Evaluation Functions

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [15]:
def rmse(predictions, actual):
    return np.sqrt(np.mean(np.square(predictions - actual)))

In [16]:
def mae(predictions, actual):
    return np.mean(np.abs(predictions - actual))

In [14]:
from sklearn.model_selection import cross_validate

models = [(RandomForestRegressor, {"random_state": random_state}), (SVR, {}), (MLPRegressor, {"random_state": random_state})]
labels_vers = [{"bbox": "e_bbox_yf", "mask": "e_face_yf"}, {"mask": "e_bbox_yf", "bbox": "e_face_yf"}]

for i in feat_regions:
    for j in labels_vers:
        global labels
        labels = j
        features, label = get_features_and_label(ASSIGNED_COLOR_SPACE, i)
        X = df_encoded_features.loc[:,  features]
        y = df_encoded_features.loc[:, label].values
        for model, params in models:
            model = model(**params)
            scores = cross_validate(model, X, y, cv=cv_folds, scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
            print(i, labels[i], model)
            print('\tave test r2', np.mean(scores["test_r2"]))
            print('\tave test rmse', np.mean(scores["test_neg_root_mean_squared_error"]))
            print('\tave test mae', np.mean(scores["test_neg_mean_absolute_error"]))
            print('\tave train r2', np.mean(scores["train_r2"]))
            print('\tave train rmse', np.mean(scores["train_neg_root_mean_squared_error"]))
            print('\tave train mae', np.mean(scores["train_neg_mean_absolute_error"]))

raise Exception("STOP")


KeyboardInterrupt



### Random Forest

Train RF model

In [ ]:
filename = "model_dumps/rfr_base_mask_" + ASSIGNED_COLOR_SPACE + ".pkl"

if (os.path.isfile(filename)):
    rfr = joblib.load(filename) 
else:
    rfr = RandomForestRegressor(random_state = random_state)
    rfr.fit(X_train_mask, y_train_mask)
    joblib.dump(rfr, filename) 
    
rfr_pred = rfr.predict(X_test_mask)
print("RMSE:", rmse(rfr_pred, y_test_mask))
print("MAE:", mae(rfr_pred, y_test_mask))

In [ ]:
filename = "model_dumps/rfr_base_bbox_" + ASSIGNED_COLOR_SPACE + ".pkl"

if (os.path.isfile(filename)):
    rfr = joblib.load(filename) 
else:
    rfr = RandomForestRegressor(random_state = random_state)
    rfr.fit(X_train_bbox, y_train_bbox)
    joblib.dump(rfr, filename) 
    
rfr_pred = rfr.predict(X_test_bbox)
print("RMSE:", rmse(rfr_pred, y_test_bbox))
print("MAE:", mae(rfr_pred, y_test_bbox))

### Support Vector Machine

In [ ]:
filename = "model_dumps/svr_base_mask_" + ASSIGNED_COLOR_SPACE + ".pkl"
if (os.path.isfile(filename)):
    svr = joblib.load(filename) 
else:
    svr = SVR()
    svr.fit(X_train_mask, y_train_mask)
    joblib.dump(svr, filename) 
    
svr_pred = svr.predict(X_test_mask)
print("RMSE:", rmse(svr_pred, y_test_mask))
print("MAE:", mae(svr_pred, y_test_mask))

In [ ]:
filename = "model_dumps/svr_base_bbox_" + ASSIGNED_COLOR_SPACE + ".pkl"

if (os.path.isfile(filename)):
    svr = joblib.load(filename) 
else:
    svr = SVR()
    svr.fit(X_train_bbox, y_train_bbox)
    joblib.dump(svr, filename)
    
svr_pred = svr.predict(X_test_bbox)
print("RMSE:", rmse(svr_pred, y_test_bbox))
print("MAE:", mae(svr_pred, y_test_bbox))

### Multilayer Perceptrons



In [ ]:
filename = "model_dumps/mpr_base_mask_" + ASSIGNED_COLOR_SPACE + ".pkl"

if (os.path.isfile(filename)):
    mpr = joblib.load(filename) 
else:
    mpr = MLPRegressor(random_state = random_state)
    mpr.fit(X_train_mask, y_train_mask)
    joblib.dump(mpr, filename) 
    
mpr_pred = mpr.predict(X_test_mask)
print("RMSE:", rmse(mpr_pred, y_test_mask))
print("MAE:", mae(mpr_pred, y_test_mask))

In [ ]:
filename = "model_dumps/mpr_base_bbox_e" + ASSIGNED_COLOR_SPACE + ".pkl"

if (os.path.isfile(filename)):
    mpr = joblib.load(filename) 
else:
    mpr = MLPRegressor(random_state = random_state)
    mpr.fit(X_train_bbox, y_train_bbox)
    joblib.dump(mpr, filename) 

mpr_pred = mpr.predict(X_test_bbox)
print("RMSE:", rmse(mpr_pred, y_test_bbox))
print("MAE:", mae(mpr_pred, y_test_bbox))

___

In [ ]:
raise Exception("STOP HERE")

## Feature selection and Hyperparameter tuning



In [17]:
CHOSEN_COLOR_SPACE = "HSV" #<-- pick a colorspace
CHOSEN_REGION = "bbox"

features, label = get_features_and_label(CHOSEN_COLOR_SPACE, CHOSEN_REGION)

X_features =  df_encoded_features.loc[:,  features]
y_features = df_encoded_features.loc[:, label].values #<-- pick label

X_train, X_test, y_train, y_test = train_test_split(X_features, y_features, test_size = 0.2, random_state=random_state)

In [18]:
# from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.base import clone as clone_model

In [26]:
models = [(RandomForestRegressor, {"random_state": random_state}), (SVR, {}), (MLPRegressor, {"random_state": random_state})]
for model, params in models:
    model = model(**params)
    model.fit(X_train, y_train)
    print(model)
    print('Test')
    y_pred = model.predict(X_test)
    print("RMSE: ", rmse(y_test, y_pred))
    print("MAE: ", mae(y_test, y_pred))

    print('Train')
    y_pred = model.predict(X_train)
    print("RMSE: ", rmse(y_train, y_pred))
    print("MAE: ", mae(y_train, y_pred))


RandomForestRegressor(random_state=100)
Test
RMSE:  0.06288397111743395
MAE:  0.041782030478395064
Train
RMSE:  0.024251129710043822
MAE:  0.015821292325056423
SVR()
Test
RMSE:  0.06715821107292488
MAE:  0.04843515171837297
Train
RMSE:  0.0679002918579933
MAE:  0.04889661486263617
MLPRegressor(random_state=100)
Test
RMSE:  0.06332630578147068
MAE:  0.04206493978021651
Train
RMSE:  0.06421700942860206
MAE:  0.042568990918430076


### Random Forest

**Grid Search**

```
param_space = [
    {
        "n_estimators": list(range(100, 351, 50)),
        "criterion": ["squared_error", "absolute_error", "friedman_mse", "poisson"],
        "max_depth": [None, 1, 10, 30, 80, 150],
        "min_samples_split": [2, 4, 8, 16, 32],
        "min_samples_leaf": [1, 5, 10, 20],
        "max_features": [None, "sqrt", "log2", 0.25, 0.5, 0.75],
        "max_leaf_nodes": [None, 50, 100, 300],
        "min_impurity_decrease": [0.0, 0.5],
        "bootstrap": [False, True],
        "ccp_alpha": [0.0, 0.5, 1.0],
    }
]
```

```
rfr = RandomForestRegressor()
gs_rfr = GridSearchCV(rfr, param_space, n_jobs=-1, cv=cv_folds)
gs_rfr.fit(X_train, y_train)
```

```
print('Best parameters found:\n', gs_svr.best_params_)
y_pred = gs_rfr.predict(X_val)
print("Best accuracy: ", rmse(y_val, y_pred))
print(classification_report(y_val, y_pred))
```

**Bayesian Optimization**

In [ ]:
search_space = {
    "n_estimators": Integer(100, 350),
    "criterion": Categorical(["squared_error", "absolute_error", "friedman_mse", "poisson"]),
    "max_depth": Integer(1, 300),
    "min_samples_split": Integer(2, 32),
    "min_samples_leaf": Integer(1, 20),
    "max_features": Categorical([None, "sqrt", "log2", 0.25, 0.5, 0.75]),
    "max_leaf_nodes": Integer(50, 300),
    "min_impurity_decrease": Real(0.0, 2.0),
    "bootstrap": Categorical([False, True]),
    "ccp_alpha": Real(0.0, 2.0),
}

In [ ]:
rfr = RandomForestRegressor()
bo_rfr = BayesSearchCV(rfr, search_space, n_iter=50, n_jobs=-1, cv=cv_folds, random_state=random_state, verbose=2)
bo_rfr.fit(X_train, y_train)

joblib.dump(bo_rfr, "model_dumps/rfr_tuned_" + CHOSEN_COLOR_SPACE + "_" + CHOSEN_REGION + ".pkl") 

In [ ]:
print('Best parameters found:\n', bo_rfr.best_params_)

print('Test')
y_pred = bo_rfr.predict(X_test)
print("RMSE: ", rmse(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

print('Train')
y_pred = bo_rfr.predict(X_train)
print("RMSE: ", rmse(y_train, y_pred))
print("MAE: ", mae(y_train, y_pred))

### Feature Selection

In [ ]:
rfr = RandomForestRegressor()
pipe = Pipeline([('selector', SelectKBest(f_regression)), ('rfr', rfr)])

search_space = {
    "selector__k": Integer(X_train.shape[1] // 2, X_train.shape[1] - 1),
    "rfr__n_estimators": Integer(100, 350),
    "rfr__criterion": Categorical(["squared_error", "absolute_error", "friedman_mse", "poisson"]),
    "rfr__max_depth": Integer(1, 300),
    "rfr__min_samples_split": Integer(2, 32),
    "rfr__min_samples_leaf": Integer(1, 20),
    "rfr__max_features": Categorical([None, "sqrt", "log2", 0.25, 0.5, 0.75]),
    "rfr__max_leaf_nodes": Integer(50, 300),
    "rfr__min_impurity_decrease": Real(0.0, 2.0),
    "rfr__bootstrap": Categorical([False, True]),
    "rfr__ccp_alpha": Real(0.0, 2.0),
}

bo_rfr = BayesSearchCV(pipe, search_space, n_iter=75, n_jobs=-1, cv=cv_folds, random_state=random_state)
bo_rfr.fit(X_train, y_train)
joblib.dump(bo_rfr, "model_dumps/rfr_tunedfs_bo_" + CHOSEN_COLOR_SPACE + "_" + CHOSEN_REGION + ".pkl") 

selected_feat = bo_rfr.best_estimator_.named_steps["selector"].get_support()
best_rfr = clone_model(bo_rfr.best_estimator_)
best_rfr.fit(X_train.loc[:, selected_feat], y_train)
joblib.dump(best_rfr, "model_dumps/rfr_tunedfs_" + CHOSEN_COLOR_SPACE + "_" + CHOSEN_REGION + ".pkl") 

In [ ]:
print('Best params:\n', bo_rfr.best_params_)
print('Best features found:\n', X_train.columns[selected_feat])

print("Test")
y_pred = best_rfr.predict(X_test.loc[:, selected_feat])
print("RMSE: ", rmse(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

print("Train")
y_pred = best_rfr.predict(X_train.loc[:, selected_feat])
print("RMSE: ", rmse(y_train, y_pred))
print("MAE: ", mae(y_train, y_pred))

### Support Vector Machine

**Grid Search**

```
param_space = [
    {
        "kernel": "poly",
        "degree": [3, 5, 10, 15, 20],
        "gamma": ["auto", "scale"],
        "coef0": [0.0, 2.5, 5.0],
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "C": [0.0001, 0.01, 1.0, 100.0, 1000.0],
        "epsilon": [0.05, 0.1, .5],
        "shrinking": [False, True],
        "max_iter": [-1, 100, 500, 1000],
    },
    {
        "kernel": "rbf",
        "gamma": ["auto", "scale"],
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "C": [0.0001, 0.01, 1.0, 100.0, 1000.0],
        "epsilon": [0.05, 0.1, .5],
        "shrinking": [False, True],
        "max_iter": [-1, 100, 500, 1000],
    },
    {
        "kernel": "sigmoid",
        "gamma": ["auto", "scale"],
        "coef0": [0.0, 2.5, 5.0],
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "C": [0.0001, 0.01, 1.0, 100.0, 1000.0],
        "epsilon": [0.05, 0.1, .5],
        "shrinking": [False, True],
        "max_iter": [-1, 100, 500, 1000],
    },
    {
        "kernel": ["linear"],
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "C": [0.0001, 0.01, 1.0, 100.0, 1000.0],
        "epsilon": [0.05, 0.1, .5],
        "shrinking": [False, True],
        "max_iter": [-1, 100, 500, 1000],
    }
]
```

```
svr = SVR()
gs_svr = GridSearchCV(svr, param_space, n_jobs=-1, cv=cv_folds)
gs_svr.fit(X_train, y_train)
```

```
print('Best parameters found:\n', gs_svr.best_params_)
y_pred = gs_svr.predict(X_val)
print("Best accuracy: ", rmse(y_val, y_pred))
print(classification_report(y_val, y_pred))
```

**Bayesian Optimization**

In [ ]:
search_space = {
    "kernel": Categorical(["linear", "poly", "rbf", "sigmoid"]),
    "degree": Integer(3, 20),
    "gamma": Categorical(["auto", "scale"]),
    "coef0": Real(0.0, 5.0),
    "tol": Real(0.0001, 0.1),
    "C": Real(0.0001, 1000.0),
    "epsilon": Real(0.05, .5),
    "shrinking": Categorical([False, True]),
    "max_iter": Integer(100, 5000),
}

In [ ]:
svr = SVR()
bo_svr = BayesSearchCV(svr, search_space, n_iter=50, n_jobs=-1, cv=cv_folds, random_state=random_state, verbose=2)
bo_svr.fit(X_train, y_train)

joblib.dump(bo_svr, "model_dumps/svr_tuned_" + CHOSEN_COLOR_SPACE + "_" + CHOSEN_REGION + ".pkl") 

In [ ]:
print('Best parameters found:\n', bo_svr.best_params_)

print('Test')
y_pred = bo_svr.predict(X_test)
print("RMSE: ", rmse(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

print('Train')
y_pred = bo_svr.predict(X_train)
print("RMSE: ", rmse(y_train, y_pred))
print("MAE: ", mae(y_train, y_pred))

### Feature Selection

In [ ]:
svr = SVR()
pipe = Pipeline([('selector', SelectKBest(f_regression)), ('svr', svr)])

search_space = {
    "selector__k": Integer(X_train.shape[1] // 2, X_train.shape[1] - 1),
    "svr__kernel": Categorical(["linear", "poly", "rbf", "sigmoid"]),
    "svr__degree": Integer(3, 20),
    "svr__gamma": Categorical(["auto", "scale"]),
    "svr__coef0": Real(0.0, 5.0),
    "svr__tol": Real(0.0001, 0.1),
    "svr__C": Real(0.0001, 1000.0),
    "svr__epsilon": Real(0.05, .5),
    "svr__shrinking": Categorical([False, True]),
    "svr__max_iter": Integer(100, 5000),
}

bo_svr = BayesSearchCV(pipe, search_space, n_iter=75, n_jobs=-1, cv=cv_folds, random_state=random_state)
bo_svr.fit(X_train, y_train)
joblib.dump(bo_svr, "model_dumps/svr_tunedfs_bo_" + CHOSEN_COLOR_SPACE + "_" + CHOSEN_REGION + ".pkl") 

selected_feat = bo_svr.best_estimator_.named_steps["selector"].get_support()
best_svr = clone_model(bo_svr.best_estimator_)
best_svr.fit(X_train.loc[:, selected_feat], y_train)
joblib.dump(best_svr, "model_dumps/svr_tunedfs_" + CHOSEN_COLOR_SPACE + "_" + CHOSEN_REGION + ".pkl") 

In [ ]:
print('Best params:\n', bo_svr.best_params_)
print('Best features found:\n', X_train.columns[selected_feat])

print("Test")
y_pred = best_svr.predict(X_test.loc[:, selected_feat])
print("RMSE: ", rmse(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

print("Train")
y_pred = best_svr.predict(X_train.loc[:, selected_feat])
print("RMSE: ", rmse(y_train, y_pred))
print("MAE: ", mae(y_train, y_pred))

### Multilayer Perceptron

**Grid Search**

```
param_space = [
    {
        "solver": "lbfgs",
        "hidden_layer_sizes": [(100,), (50, 50,), (50, 25, 25,)], # pick better ones
        "activation": ["identity", "logistic", "tanh", "relu"],
        "alpha": [0.00001, 0.0001, 0.001],
        "max_iter": [200, 500, 1000],
        "random_state": random_state,
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "max_fun": [10000, 15000],
    },
    {
        "solver": "adam",
        "hidden_layer_sizes": [(100,), (50, 50,), (50, 25, 25,)], # pick better ones
        "activation": ["identity", "logistic", "tanh", "relu"],
        "alpha": [0.00001, 0.0001, 0.001],
        "max_iter": [200, 500, 1000],
        "random_state": random_state,
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "batch_size": ['auto', n_samples // 5, n_samples // 10],
        "learning_rate_init": [0.0005, 0.001, 0.005],
        "shuffle": [False, True],
        "early_stopping": [False, True],
        "validation_fraction": [0.1, 0.15],
        "n_iter_no_change": [10, 15],
        # not sure abt these values, should we be changing these?
        "beta_1": [0.75, 0.9],
        "beta_2": [0.85, 0.999],
        "epsilon": [1e-07, 1e-08],
    },
    {
        "solver": "sgd",
        "hidden_layer_sizes": [(100,), (50, 50,), (50, 25, 25,)], # pick better ones
        "activation": ["identity", "logistic", "tanh", "relu"],
        "alpha": [0.00001, 0.0001, 0.001],
        "max_iter": [200, 500, 1000],
        "random_state": random_state,
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "batch_size": ['auto', n_samples // 5, n_samples // 10],
        "learning_rate_init": [0.0005, 0.001, 0.005],
        "shuffle": [False, True],
        "early_stopping": [False, True],
        "validation_fraction": [0.1, 0.15],
        "n_iter_no_change": [10, 15],
        "learning_rate": ["constant", "invscaling", "adaptive"],
        "momentum": [0.75, 0.9],
        "nesterovs_momentum": [False, True],
    }
]
```

```
mpr = MLPRegressor()
gs_mpr = GridSearchCV(mpr, param_space, n_jobs=-1, cv=cv_folds)
gs_mpr.fit(X_train, y_train)
```

```
print('Best parameters found:\n', gs_mpr.best_params_)
y_pred = gs_mpr.predict(X_val)
print("Best accuracy: ", rmse(y_val, y_pred))
print(classification_report(y_val, y_pred))
```

**Bayesian Optimization**

In [ ]:
hidden_sz = X_train.shape[1] * 2 // 3 + 1
search_space = {
    "activation": Categorical(["identity", "logistic", "tanh", "relu"]),
    "solver": Categorical(["lbfgs", "adam", "sgd"]),
    "alpha": Real(0.00001, 0.001),
    "learning_rate": Categorical(["constant", "invscaling", "adaptive"]),
    "learning_rate_init": Real(0.0005, 0.005),
    "max_iter": Integer(200, 1000),
    "shuffle": Categorical([False, True]),
    "tol": Real(0.0001, 0.1),
    "momentum": Real(0.75, 0.9),
    "nesterovs_momentum": Categorical([False, True]),
    "early_stopping": Categorical([False, True]),
    "validation_fraction": Real(0.1, 0.15),
    "beta_1": Real(0.75, 0.9),
    "beta_2": Real(0.85, 0.999),
    "epsilon": Real(1e-08, 1e-07),
    "n_iter_no_change": Integer(10, 15),
    "max_fun": Integer(10000, 15000),
}
print(hidden_sz)

In [ ]:
mpr = MLPRegressor(random_state=random_state, hidden_layer_sizes=(hidden_sz, hidden_sz * 2 // 3, hidden_sz * 4 // 9))
bo_mpr = BayesSearchCV(mpr, search_space, n_iter=50, n_jobs=-1, cv=cv_folds, random_state=random_state, verbose=2)
bo_mpr.fit(X_train, y_train)

joblib.dump(bo_mpr, "model_dumps/mpr_tuned_" + CHOSEN_COLOR_SPACE + "_" + CHOSEN_REGION + ".pkl") 

In [ ]:
print('Best parameters found:\n', bo_mpr.best_params_)

print("Test")
y_pred = bo_mpr.predict(X_test)
print("Best accuracy: ", rmse(y_test, y_pred))
print(classification_report(y_test, y_pred))

print("Train")
y_pred = bo_mpr.predict(X_train)
print("Best accuracy: ", rmse(y_train, y_pred))
print(classification_report(y_train, y_pred))

### Feature Selection

In [ ]:
mpr = MLPRegressor(random_state=random_state, hidden_layer_sizes=(hidden_sz, hidden_sz * 2 // 3, hidden_sz * 4 // 9))
pipe = Pipeline([('selector', SelectKBest(f_regression)), ('mpr', mpr)])

search_space = {
    "selector__k": Integer(X_train.shape[1] // 2, X_train.shape[1] - 1),
    "mpr__activation": Categorical(["identity", "logistic", "tanh", "relu"]),
    "mpr__solver": Categorical(["lbfgs", "adam", "sgd"]),
    "mpr__alpha": Real(0.00001, 0.001),
    "mpr__learning_rate": Categorical(["constant", "invscaling", "adaptive"]),
    "mpr__learning_rate_init": Real(0.0005, 0.005),
    "mpr__max_iter": Integer(200, 1000),
    "mpr__shuffle": Categorical([False, True]),
    "mpr__tol": Real(0.0001, 0.1),
    "mpr__momentum": Real(0.75, 0.9),
    "mpr__nesterovs_momentum": Categorical([False, True]),
    "mpr__early_stopping": Categorical([False, True]),
    "mpr__validation_fraction": Real(0.1, 0.15),
    "mpr__beta_1": Real(0.75, 0.9),
    "mpr__beta_2": Real(0.85, 0.999),
    "mpr__epsilon": Real(1e-08, 1e-07),
    "mpr__n_iter_no_change": Integer(10, 15),
    "mpr__max_fun": Integer(10000, 15000),
}

bo_mpr = BayesSearchCV(pipe, search_space, n_iter=76, n_jobs=-1, cv=cv_folds, random_state=random_state)
bo_mpr.fit(X_train, y_train)
joblib.dump(bo_mpr, "model_dumps/mpr_tunedfs_bo_" + CHOSEN_COLOR_SPACE + "_" + CHOSEN_REGION + ".pkl") 

selected_feat = bo_mpr.best_estimator_.named_steps["selector"].get_support()
best_mpr = clone_model(bo_mpr.best_estimator_)
best_mpr.fit(X_train.loc[:, selected_feat], y_train)
joblib.dump(best_mpr, "model_dumps/mpr_tunedfs_" + CHOSEN_COLOR_SPACE + "_" + CHOSEN_REGION + ".pkl") 

In [ ]:
print('Best params:\n', bo_mpr.best_params_)
print('Best features found:\n', X_train.columns[selected_feat])

print("Test")
y_pred = best_mpr.predict(X_test.loc[:, selected_feat])
print("RMSE: ", rmse(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

print("Train")
y_pred = best_mpr.predict(X_train.loc[:, selected_feat])
print("RMSE: ", rmse(y_train, y_pred))
print("MAE: ", mae(y_train, y_pred))

In [19]:
mlp_params = dict([('activation', 'relu'), ('alpha', 0.0006077414104241369), ('beta_1', 0.75), ('beta_2', 0.9174938922030286), ('early_stopping', True), ('epsilon', 1e-08), ('learning_rate', 'invscaling'), ('learning_rate_init', 0.0005), ('max_fun', 10000), ('max_iter', 1000), ('momentum', 0.75), ('n_iter_no_change', 15), ('nesterovs_momentum', True), ('shuffle', False), ('solver', 'adam'), ('tol', 0.0001), ('validation_fraction', 0.15)])
use_features = ['w', 'h', 'x', 'y', 'H_HSV_BIN_bbox_0', 'H_HSV_BIN_bbox_2',
       'H_HSV_BIN_bbox_7', 'H_HSV_BIN_bbox_9', 'H_HSV_BIN_bbox_10',
       'H_HSV_BIN_bbox_12', 'H_HSV_BIN_bbox_13', 'H_HSV_BIN_bbox_14',
       'H_HSV_BIN_bbox_18', 'H_HSV_BIN_bbox_22', 'H_HSV_BIN_bbox_24',
       'S_HSV_BIN_bbox_2', 'S_HSV_BIN_bbox_3', 'S_HSV_BIN_bbox_7',
       'S_HSV_BIN_bbox_8', 'S_HSV_BIN_bbox_9', 'S_HSV_BIN_bbox_10',
       'S_HSV_BIN_bbox_11', 'S_HSV_BIN_bbox_15', 'S_HSV_BIN_bbox_16',
       'S_HSV_BIN_bbox_17', 'S_HSV_BIN_bbox_18', 'S_HSV_BIN_bbox_19',
       'S_HSV_BIN_bbox_20', 'S_HSV_BIN_bbox_21', 'S_HSV_BIN_bbox_22',
       'S_HSV_BIN_bbox_23', 'S_HSV_BIN_bbox_24', 'V_HSV_BIN_bbox_2',
       'V_HSV_BIN_bbox_3', 'V_HSV_BIN_bbox_4', 'V_HSV_BIN_bbox_5',
       'V_HSV_BIN_bbox_6', 'V_HSV_BIN_bbox_7', 'V_HSV_BIN_bbox_8',
       'V_HSV_BIN_bbox_9', 'V_HSV_BIN_bbox_10', 'V_HSV_BIN_bbox_11',
       'V_HSV_BIN_bbox_12', 'V_HSV_BIN_bbox_13', 'V_HSV_BIN_bbox_14',
       'V_HSV_BIN_bbox_15', 'V_HSV_BIN_bbox_16', 'V_HSV_BIN_bbox_17',
       'V_HSV_BIN_bbox_19', 'V_HSV_BIN_bbox_20', 'V_HSV_BIN_bbox_21',
       'V_HSV_BIN_bbox_22', 'V_HSV_BIN_bbox_23', 'V_HSV_BIN_bbox_24',
       'V_HSV_BIN_bbox_25', 'LBP_BIN_bbox_0', 'LBP_BIN_bbox_1',
       'LBP_BIN_bbox_2', 'LBP_BIN_bbox_3', 'LBP_BIN_bbox_4', 'LBP_BIN_bbox_5',
       'LBP_BIN_bbox_6', 'LBP_BIN_bbox_7', 'LBP_BIN_bbox_8', 'LBP_BIN_bbox_9',
       'LBP_BIN_bbox_11', 'LBP_BIN_bbox_12', 'LBP_BIN_bbox_13',
       'LBP_BIN_bbox_14', 'LBP_BIN_bbox_15', 'LBP_BIN_bbox_16',
       'LBP_BIN_bbox_18', 'LBP_BIN_bbox_20', 'LBP_BIN_bbox_21',
       'LBP_BIN_bbox_22', 'LBP_BIN_bbox_23', 'LBP_BIN_bbox_24',
       'LBP_BIN_bbox_25', 'SOBELX_BIN_bbox_9', 'SOBELX_BIN_bbox_10',
       'SOBELY_BIN_bbox_9', 'SOBELY_BIN_bbox_10', 'SOBEL_BIN_bbox_9',
       'SOBEL_BIN_bbox_10']

new_mlp = MLPRegressor(random_state=random_state, **mlp_params)

search_space = {
    "selector__k": Integer(1, X_train.shape[1] - 1),
    "layers__sz0": Integer(1, X_train.shape[1] - 1),
    "layers__sz1": Integer(1, X_train.shape[1] - 1),
    "layers__sz2": Integer(1, X_train.shape[1] - 1),
    "layers__sz3": Integer(1, X_train.shape[1] - 1),
    "layers__d": Integer(1, 4),
}

from sklearn.base import BaseEstimator, TransformerMixin
class Layers(BaseEstimator, TransformerMixin):
    def __init__(self, d=4, *args, **kwargs):
        for i in range(d):
            if i < len(args):
                setattr(self, "sz" + str(i), args[i])
            else:
                setattr(self, "sz" + str(i), 1)
                
        self.set_params(**kwargs)
        self.d = d
    
    def get_params(self, deep=False):
        sizes = {"sz" + str(i): getattr(self, "sz" + str(i)) for i in range(self.d)}
        sizes["d"] = self.d
        return sizes
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        global new_mlp
        new_mlp.set_params(hidden_layer_sizes=tuple(getattr(self, "sz" + str(i)) for i in range(self.d)))
        return X
    
pipe = Pipeline([('selector', SelectKBest(f_regression)), ('layers', Layers()), ('mpr', new_mlp)])
search = BayesSearchCV(pipe, search_space, n_iter=15, n_jobs=-1, cv=cv_folds, random_state=random_state)

In [33]:
new_mlp = MLPRegressor(random_state=random_state, **mlp_params)

search_space = {
    "selector__k": Integer(X_train.shape[1] // 3, X_train.shape[1] - 1),
    "layers__sz": Integer(1, X_train.shape[1] - 1),
    "layers__d": Integer(1, 3),
}

from sklearn.base import BaseEstimator, TransformerMixin
class Layers(BaseEstimator, TransformerMixin):
    def __init__(self, d=4, sz=100):
    def get_params(self, deep=False):
        return {"sz": self.sz, "d": self.d}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        global new_mlp
        new_mlp.set_params(hidden_layer_sizes=tuple(self.sz * 2 // 3 for i in range(self.d)))
        return X
    
pipe = Pipeline([('selector', SelectKBest(f_regression)), ('layers', Layers()), ('mpr', new_mlp)])
search = BayesSearchCV(pipe, search_space, n_iter=15, n_jobs=-1, cv=cv_folds, random_state=random_state)

In [34]:
search.fit(X_train, y_train)
selected_feat = search.best_estimator_.named_steps["selector"].get_support()
best_mpr = clone_model(search.best_estimator_)
best_mpr.fit(X_train.loc[:, selected_feat], y_train)

Pipeline(steps=[('selector',
                 SelectKBest(k=56,
                             score_func=<function f_regression at 0x00000228B860C670>)),
                ('layers', Layers(d=1, sz=148)),
                ('mpr',
                 MLPRegressor(alpha=0.0006077414104241369, beta_1=0.75,
                              beta_2=0.9174938922030286, early_stopping=True,
                              learning_rate='invscaling',
                              learning_rate_init=0.0005, max_fun=10000,
                              max_iter=1000, momentum=0.75, n_iter_no_change=15,
                              random_state=100, shuffle=False,
                              validation_fraction=0.15))])

In [35]:
y_train.mean()
y_train.std()

0.08129015324047521

In [36]:
#pipe.fit(X_train, y_train)
#pipe.named_steps["selector"].get_params()
#pipe.set_params(**{"layers__sz": 1, "layers__d": 1})
#pipe.named_steps["layers"].get_params()
#pipe.get_params()

In [1]:
best_mpr.fit(X_train, y_train)
print('Best params:\n', search.best_params_)
print('Best features found:\n', X_train.columns[selected_feat])

print("Test")
y_pred = best_mpr.predict(X_test)
print("RMSE: ", rmse(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

print("Train")
y_pred = best_mpr.predict(X_train)
print("RMSE: ", rmse(y_train, y_pred))
print("MAE: ", mae(y_train, y_pred))

NameError: name 'best_mpr' is not defined

In [26]:
best_params = dict([('mpr__activation', 'relu'), ('mpr__alpha', 0.0006077414104241369), ('mpr__beta_1', 0.75), ('mpr__beta_2', 0.9174938922030286), ('mpr__early_stopping', True), ('mpr__epsilon', 1e-08), ('mpr__learning_rate', 'invscaling'), ('mpr__learning_rate_init', 0.0005), ('mpr__max_fun', 10000), ('mpr__max_iter', 1000), ('mpr__momentum', 0.75), ('mpr__n_iter_no_change', 15), ('mpr__nesterovs_momentum', True), ('mpr__shuffle', False), ('mpr__solver', 'adam'), ('mpr__tol', 0.0001), ('mpr__validation_fraction', 0.15), ('mpr__hidden_layer_sizes', (78, 78,)), ('selector__k', 75)])

pipe = Pipeline([('selector', SelectKBest(f_regression)), ('mpr', MLPRegressor(random_state=random_state))])
pipe.set_params(**best_params)
pipe.fit(X_train, y_train)
joblib.dump(pipe, "best_model_so_far.pkl")



Test


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- H_HSV_BIN_bbox_0
- H_HSV_BIN_bbox_1
- H_HSV_BIN_bbox_10
- H_HSV_BIN_bbox_11
- H_HSV_BIN_bbox_12
- ...


In [27]:
print("Test")
y_pred = pipe.predict(X_test)
print("RMSE: ", rmse(y_test, y_pred))
print("MAE: ", mae(y_test, y_pred))

print("Train")
y_pred = pipe.predict(X_train)
print("RMSE: ", rmse(y_train, y_pred))
print("MAE: ", mae(y_train, y_pred))

Test
RMSE:  0.06421086009038143
MAE:  0.04411909984089898
Train
RMSE:  0.06299828271531498
MAE:  0.043584044092813176


## Evaluation

#